In [ ]:
!pip install wikipedia
!pip install surprise
!pip install ipyplot

In [34]:
import os
import pprint
import operator
import numpy as np
import pandas as pd
from os import listdir
from sklearn.metrics.pairwise import pairwise_distances
import pandas as pd
import plotly.express as px
from sklearn.feature_extraction.text import TfidfVectorizer
import wikipedia
import numpy as np
from tqdm.notebook import tqdm
import re
import os
from sklearn.metrics.pairwise import linear_kernel
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_selection import VarianceThreshold
import requests
from io import BytesIO
import nltk
import ipyplot


        WARNING! Google Colab Environment detected!
        You might encounter issues while running in Google Colab environment.
        If images are not displaying properly please try setting `force_b64` param to `True`.
        


In [42]:
from tqdm.auto import tqdm
tqdm.pandas()

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
%cd '/content/drive/MyDrive/Colab Notebooks'

/content/drive/MyDrive/Colab Notebooks


In [9]:
#Information about the data
info = pd.read_csv("CAERS/ml-100k/u.info", sep=" ", header=None)
info.columns=['Counts', 'Type']
info

,Counts,Type
0,943,users
1,1682,items
2,100000,ratings


In [12]:
#Types of genres
genre = pd.read_csv('CAERS/ml-100k/u.genre', sep="|", encoding='latin-1', header=None)
genre.drop(genre.columns[1], axis=1, inplace=True)
genre.columns = ['Genres']
genre_list = list(genre['Genres'])
genre_list

['unknown',
 'Action',
 'Adventure',
 'Animation',
 "Children's",
 'Comedy',
 'Crime',
 'Documentary',
 'Drama',
 'Fantasy',
 'Film-Noir',
 'Horror',
 'Musical',
 'Mystery',
 'Romance',
 'Sci-Fi',
 'Thriller',
 'War',
 'Western']

In [14]:
#Types of occupations
occupation = pd.read_csv('CAERS/ml-100k/u.occupation', sep="|", encoding='latin-1', header=None)
occupation.columns = ['Occupations']
occupation_list = list(occupation['Occupations'])
occupation_list

['administrator',
 'artist',
 'doctor',
 'educator',
 'engineer',
 'entertainment',
 'executive',
 'healthcare',
 'homemaker',
 'lawyer',
 'librarian',
 'marketing',
 'none',
 'other',
 'programmer',
 'retired',
 'salesman',
 'scientist',
 'student',
 'technician',
 'writer']

In [15]:
#Load the Ratings data
data = pd.read_csv('CAERS/ml-100k/u.data', sep="\t", header=None)
data.columns = ['user id', 'movie id', 'rating', 'timestamp']
data.head()

,user id,movie id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [16]:
#Load the Movies data
item = pd.read_csv('CAERS/ml-100k/u.item', sep="|", encoding='latin-1', header=None)
item.columns = ['movie id', 'movie title' ,'release date','video release date', 'IMDb URL', 'unknown', 'Action',
                'Adventure', 'Animation', 'Children\'s', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir',
                'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']
item.head()

,movie id,movie title,release date,video release date,IMDb URL,unknown,Action,Adventure,Animation,Children's,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [17]:
#Load the User data
user = pd.read_csv('CAERS/ml-100k/u.user', sep="|", encoding='latin-1', header=None)
user.columns = ['user id', 'age', 'gender', 'occupation', 'zip code']
user.head()

,user id,age,gender,occupation,zip code
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213


In [18]:
#merge the 'data' table with 'user' table
data_user = pd.merge(data[['user id', 'movie id', 'rating']], user[['user id', 'occupation']], on='user id')
data_user.drop(columns = ['user id'], inplace=True)

#merge the 'Data_User' dataframe with 'Item' dataframe to get each rating, occupation of user and movie title
data_user_item = pd.merge(data_user[['movie id', 'rating', 'occupation']], item[['movie id', 'movie title']], on='movie id')
data_user_item.drop(columns = ['movie id'], inplace=True)

#group the data by occupation and movie title and sort with decreasing average ratings
data_user_item_sorted = data_user_item.groupby(['occupation', 'movie title'], as_index=False)['rating'].mean().sort_values('rating', ascending=False)

#group data by occupation, movie title and select top 3 movies for each occupation
top_3_occ = data_user_item_sorted.groupby(['occupation']).head(3).sort_values(['occupation', 'movie title'], ascending=[True, True]).reset_index()
top_3_occ.drop(['index'], axis=1, inplace=True)

top_3_occ

,occupation,movie title,rating
0,administrator,When We Were Kings (1996),5.0
1,administrator,"Winter Guest, The (1997)",5.0
2,administrator,"World of Apu, The (Apur Sansar) (1959)",5.0
3,artist,"39 Steps, The (1935)",5.0
4,artist,"Adventures of Pinocchio, The (1996)",5.0
...,...,...,...
58,technician,Basquiat (1996),5.0
59,technician,Beautiful Thing (1996),5.0
60,writer,Faster Pussycat! Kill! Kill! (1965),5.0
61,writer,Faust (1994),5.0


In [ ]:
#merge the 'data' table with the 'item' table
data_item = pd.merge(data[['user id', 'movie id', 'rating']], item, on='movie id')
data_item.drop(columns = ['user id', 'movie id', 'release date', 'video release date', 'IMDb URL'], inplace=True)

#For each genre get the top 3 movies by average rating
top_3_genre = pd.DataFrame()
for gen in genre_list:
    g_r = data_item[data_item[gen] == 1]
    new_gen = pd.DataFrame(g_r.groupby(['movie title'], as_index=False)['rating'].mean().sort_values(['rating', 'movie title'], ascending=[False, True]).head(3))
    new_gen.insert(0, 'genre', gen)
    top_3_genre = top_3_genre.append(new_gen, ignore_index=True)

top_3_genre

In [31]:
movies_df = top_3_genre
movies_df.head()

,genre,movie title,rating
0,unknown,unknown,3.444444
1,unknown,Good Morning (1971),1.000000
2,Action,Star Wars (1977),4.358491
3,Action,"Godfather, The (1972)",4.283293
4,Action,Raiders of the Lost Ark (1981),4.252381


In [29]:
user.head()

,user id,age,gender,occupation,zip code
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213


In [35]:
p= wikipedia.page('Toy Story (1998)')
s = p.content

In [36]:
def get_wikipedia_page_name(raw_name):
    names = wikipedia.search(raw_name)
    if len(names) == 0:
      return ''
    else:
      return names[0]


In [37]:
def get_movie_plot(page_name):
    try:
      try:
        movie_page_content = str(wikipedia.page(page_name, auto_suggest=False).content)
      except wikipedia.DisambiguationError as e:
        for option in e.options:
          if 'film' in option:
            movie_page_content = str(wikipedia.page(option, auto_suggest=False).content)
        return np.nan
    except (wikipedia.PageError, KeyError):
      return np.nan
    re_groups = re.search("Plot ==(.*?)=+ [A-Z]", str(movie_page_content).replace('\n', ''))
    if re_groups:
      return re_groups.group(1)
    else:
      return np.nan

In [44]:
movies_df['wikipedia_page_name'] = movies_df['movie title'].progress_apply(lambda name: get_wikipedia_page_name(name))
movies_df['movie_plot'] = movies_df['wikipedia_page_name'].progress_apply(lambda page_name: get_movie_plot(page_name))
print(f'There are {movies_df["movie_plot"].isna().sum()} NaN movie plots')
movies_df[['movie title', 'movie_plot']].head()


  0%|          | 0/56 [00:00<?, ?it/s]

  0%|          | 0/56 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /usr/local/lib/python3.10/dist-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


There are 8 NaN movie plots


,movie title,movie_plot
0,unknown,NaN
1,Good Morning (1971),NaN
2,Star Wars (1977),"Amid a galactic civil war, Rebel Alliance spie..."
3,"Godfather, The (1972)","In 1945, the New York City Corleone family don..."
4,Raiders of the Lost Ark (1981),"In 1936, American archaeologist Indiana Jones ..."


In [45]:
user.head()

,user id,age,gender,occupation,zip code
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213


In [46]:
movies_df.head()

,genre,movie title,rating,wikipedia_page_name,movie_plot
0,unknown,unknown,3.444444,Unknown,NaN
1,unknown,Good Morning (1971),1.000000,Good Morning Starshine,NaN
2,Action,Star Wars (1977),4.358491,Star Wars (film),"Amid a galactic civil war, Rebel Alliance spie..."
3,Action,"Godfather, The (1972)",4.283293,The Godfather,"In 1945, the New York City Corleone family don..."
4,Action,Raiders of the Lost Ark (1981),4.252381,Raiders of the Lost Ark,"In 1936, American archaeologist Indiana Jones ..."


In [49]:
user.to_pickle('/content/drive/MyDrive/Colab Notebooks/CAERS/user100k.csv')
movies_df.to_pickle('/content/drive/MyDrive/Colab Notebooks/CAERS/movie100k.csv')
data.to_pickle('/content/drive/MyDrive/Colab Notebooks/CAERS/ratings100k.csv')